* https://arxiv.org/pdf/1411.4038.pdf -- FCN (2015)
* https://arxiv.org/pdf/1505.04597.pdf -- U-net (2015)
* https://arxiv.org/pdf/1511.00561.pdf -- SegNet (2016)
* https://arxiv.org/pdf/1612.01105.pdf -- PSPNet (2017)
* https://arxiv.org/pdf/1901.02446.pdf -- FPN (2019)


* https://arxiv.org/pdf/1904.04514.pdf -- HRNetV2 (2019)
* https://arxiv.org/pdf/1706.05587.pdf -- DeepLabV3+ (2017)
* https://arxiv.org/pdf/1210.5644.pdf -- CRFs (2012)

## U-Net

В базовой [статье](https://arxiv.org/pdf/1505.04597.pdf) авторы развивают идею предложенных ранее полностью сверточных нейронных сетей (fully convolutional networks). 

Сеть представляет из себя encoder-decoder архитектуру. На этапе encoder происходит извлечение признаков различных масштабов, аналогично тому, как это происходит в обычной сверточной сети для классификации. Для увеличения receptive field используются слои пуллинга, которые уменьшают разрешение изображения при прохождении его через нейронную сеть. 

На этапе decoder полученные карты признаков разворачиваются обратно до исходного разрешения при помощи операции upsampling с билинейной интерполяцией полученных карт признаков. 

Проблема такой архитектуы заключается в том, что из-за понижения разрешения теряется точность локализации признаков на картах более высокого разрешения при увеличении разрешения. Для решения данной проблемы авторы статьи предложили использовать конкатенацию карт признаков, полученных при увеличении разрешения с картами признаков соответствующего разрешения, полученными на этапе кодирования (encoding). Это позволило существенно улучшить точность локализации и, соответственно, качество итоговой сегментации.

Архитектура данной сети приведена на изображении ниже.

<img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" alt="U-Net arcitecture" width=600 />

К достоинствам данной архитектуры можно отнести то, что она может обучаться на достаточно небольшой обучающей выборке. Кроме того, для улучшения качества работы, можно использовать предобученный encoder. 

## FCN, SegNet

Полностью сверточные сети (fully-convolutional networks) -- архитектура, которая легла в основу описанного выше U-Net. Она описана с данной [статье](https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf). Ее особенностью по сравнению с архитектурами для классификации является то, что даже в голове сети используются сверточные слои вместо fully-connected. Это позволяет на выходе сети получить не один конкретный ответ, а сразу карту сегментации.

Архитектура FCN Приведена на рисунке ниже:

<img src="http://deeplearning.net/tutorial/_images/cat_segmentation.png" alt="FCN arcitecture" width=600 />

Развитием данного подхода можно также считать архитектуру SegNet. Она, так же как и описанный выше U-Net является encoder-decoder моделью, но, в отличие от U-Net, не использует признаки высокого разрешения, полученные на этапе encoder, во время генерации масок на этапе decoder. Модель подробно описывается в данной [статье](https://arxiv.org/pdf/1511.00561.pdf).

Архитектура модели приведена на изображении ниже:

<img src="https://camo.githubusercontent.com/1e568c27d091bdf210e4c71da77289dde321b47a/68747470733a2f2f6672616d617069632e6f72672f51634b4d4b4e6f53334e4e782f4861563065736c6669783650" alt="SegNet architecture" width=600 />

## PSPNet

Важны недостатком всех описанных выше архитектур является то, что они для сегментации используют признаки, полученные локально в пределах некоторой области от исходного пикселя, но не учитываю глобальных признаков всего изображения. Использование подобных признаков, по мнению авторов [статьи](https://arxiv.org/pdf/1612.01105.pdf), могло бы существенно улучшить качество полученных масок. Авторы выделяют 3 основных типа ситуаций, в которых извлечение глобальных закономерностей из итоговых карт признаков помогает точнее строить маски сегментации:

**Ошибка нелогичной классификации**. К данной ошибке приводит отсутствие представлений о классах, окружающих некоторый объект. В статье приводится изображение, на котором FCN выделяет часть лодки как автомобиль из-за визуального сходства, не принимая во внимание фактическое расположение объекта на поверхности правильно классифицированного озера

**Ошибка смежных категорий**. В датасетах могут встречаться объекты из смежных категорий (таких, как здания и небоскребы, как различные классы). FCN при этом может отнести различные части одного объекта к различным категориям, что приводит к ошибке, особенно на изображениях, на которых объекты таких категорий находятся рядом. Предложенный подход позволяет, исходя из окружения, отнести отдельный классифицируемый объект полностью к одной конкретной категории.

**Ошибка неразличимых классов**. В данном случае имеются в виду объекты, определенно относящиеся к разным классам, но имеющие очень близкий внешний вид. В примере из статьи приводились классы "подушка" и "одеяло", которые находились в одной области. В данном случае их можно различить исходя из общих представлений об изображении полностью, но практически невозможно посмотрев только на локальную область вокруг внутренней точки каждого из этих объектов.

Для получение более глобальных признаков области предлагается использовать слой *Pyramid Pooling Module*, который строит пирамиду изображений из полученных карт признаков на выходе FCN, а затем при помощи свертки 1х1 строит новые признаки, аггрегирующие в себе информацию о некоторой окрестности из каждого из слоев пирамиды. Полученные признаки затем приводятся к исходному размеру и добавляются к выходам FCN как новые признаки. Затем построение карт выполняется исходя уже из всей совокупности признаков.

Также к особенностям обучения данной архитектуры можно отнести наличие промежуточной функции потерь в процессе обучения на выходах backbone-сети. Данную функцию потерь предлагается добавить к итоговой с весом 0.4 для баланса выходов. Она помогает лучше обучить основную сеть. При тестировании модели данная функция потерь не вычисляется.

Архитектура данной сети приведена на изображении ниже.

<img src="https://miro.medium.com/max/1043/1*IxUlWP8RBtxNS1N6hyBAxA.png" alt="PSPNet architecture" width=600 />

Также к плюсам данной архитектуры можно отнести ее лучшую скорость работы по сравнению с U-Net. Однако, при решении поставленной задачи, данная архитектура должна показать худшие результаты, чем U-Net, так как она строится поверх архитектуры многоклассовой сегментации и уточняет полученные карты признаков на основании значений различных признаков.



## Feature pyramid networks (FPN)

Данная архитектура, так же, как и U-net, SegNet и некоторые другие архитектуры представляет из себя encoder-decoder архитектуру. Описание модели можно найти в следующей [статье](https://arxiv.org/pdf/1901.02446.pdf). 

Отличие от вышеуказанных архитектур в используемом декодере. В большинстве из них результат работы модели строится из выхода последнего слоя декодера. При этом частично теряются возможно важные признаки низкого разрешения, которые получаются на последних слоях энкодера. Идея данной архитектуры заключается в сохранении карт признаков в промежуточном разрешении и использовании их для дальнейшей классификации. Эти карты образуют пирамиду признаков.

Для итоговой классификации, карты признаков во всех разрешениях увеличиваются с испоьзованием билинейной интерполяции до размера максимальной из них, равного размеру входного изображения. Затем, полученные карты признаков в максимальном разрешении конкатенируются, и таким образом составляется совокупная карта признаков. Классификация строится из полученной совокупной карты признаков при помощи одного сверточного слоя. 

Архитектура данной сети показана на рисунке ниже:

<img src="https://user-images.githubusercontent.com/527241/42149076-00cf1d80-7dd5-11e8-94c6-5ac0e79d3b77.png" alt="FPN architecture" width=600 />


## HRNet (v2)

Для решения задачи сегментации, object detection, human pose estimation, в которых результатом работы является некоторый набор пространственных признаков, важным является учет не только признаков никого разрешения, полученных при понижении раразрешения изображения, но и признаков изображений в высоком разрешении. Задачу сохраниня и модификации признаков высокого разрешения на протяжении работы всей сети тавили перед собой авторы данной [статьи](https://arxiv.org/pdf/1904.04514.pdf).

Данная сеть изначально разрабатывалась, как сеть для определения поз людей на изображении (human pose estimation). Она состоит из нескольких параллельных потоков данных, которые периодически обмениваются картами признаков. Данные потоки работают с признаками в разных разрешениях. При разработке [первой версии данной модели](https://arxiv.org/pdf/1902.09212.pdf) был произведен подбор архитектуры, который привел авторов к тому, что периодический обмен признаками между потоками данных дает наилучшее качество. 

Карты признаков, полученные в результате работы каждого из потоков данных в разном разрешении приводятся к максимальному разрешению и конкатенируются. Затем, на основе итоговых карт признаков, полученных в разных разных разрешениях, строится итоговое предсказание. Здесь для получения итоговых масок используется подход, аналогичный FPN, описанным выше.

Архитектура представлена на изображении ниже. На данный момент данная модель является SOTA и показывает один из дучших результатов на различных датасетах
[$^1$](https://paperswithcode.com/sota/semantic-segmentation-on-pascal-context)
[$^2$](https://paperswithcode.com/sota/semantic-segmentation-on-ade20k)
[$^3$](https://paperswithcode.com/sota/semantic-segmentation-on-cityscapes). 

<img src="https://raw.githubusercontent.com/HRNet/HRNet-Semantic-Segmentation/master/figures/seg-hrnet.png" alt="HRNet V2 arcitecture" />

К сожалению, в ходе выполнения данной работы, времени на обучение данной модели не хватило.

## DeepLab

В данной [работе](https://arxiv.org/pdf/1606.00915v2.pdf) успешно комбинируются различные подходы к улучшению качества сегментации. Большинство из описанных подходах были предложены в более ранних статьях. Здесь будут перечислены отдельно основные идеи статьи.

**Atrous convolution**. Обычные сверточные слои извлекают информацию из некоторй окрестности целевого пикселя и по этой информации строят итоговый ответ. В большинстве архитектур receptive field -- область, влияет на результат свертки довольно низок. В большинстве архитектур используются свертки размером 3х3. Использование сверток 5х5 существенно более сложное вычислительно (25 операций против 9). При этом 2 идущие подряд свертки 3х3 в итоге захватывают область того же размера, что и одна свертка 5х5, но при этом количество операций все равно остается меньше (9 против 25)ю Atrous convolution -- способ сохранить количество операций на слой, но при этом повысить receptive field. В этих свертках используется разреженная карта весов. В ней количество весов остается тем же, но при этом имеются пропуски. между весами. Таким образом, вычислительная сложность возрастает незначительно, но при свертка покрывает большую область изображения.

<img src="https://miro.medium.com/max/1130/1*-r7CL0AkeO72MIDpjRxfog.png" alt="Atrous convolution" width=600 />

**Atrous Spatial Pyramid Pooling**. В сетях для классификации одним из способов увеличения receptive field является операция пуллинга. При этом из некоторой окрестности пикселя извлекается некоторый аггрегированый признак (максимум, среднее, и т.п.). Однако, окрестность, которая учитывается при агрегировании, является довольно маленькой. При этом, для лучшей работы алгоритма сегментации, важной задачей является извлечение признаков из довольно большой области. Для этого было предложено использовать atrous convolutions с различными расстояниями между весами. Таким образом, после данного слоя, в карте признаков будут содержаться как значения из непосредственной окрестности пикселя, так и из большой области изображения вокруг.

<img src="https://www.researchgate.net/profile/Lin_Chen228/publication/331556645/figure/fig1/AS:733487137054723@1551888369843/Illustration-of-Atrous-Spatial-Pyramid-Pooling-ASPP-Max-pooling-a-11-convolution-and.ppm" alt="ASPP" width=600 />

**Fully Connected Conditional Random Fields**. Данный алгоритм был предложен для улучшения качества работы алгоритмов сегментации до "эпохи нейронных сетей". Он представляет из себя вероятностную марковскую модель. В результате настройки, модель определяет вероятности классов для некоторого пикселя -- узла сети исходя из предсказанных основной можелью вероятностей, а также исходя из ответов модели для схожих пикселей из всего изображения и окрестности данного.

Дополнительное описание можно найти в данной [статье](https://arxiv.org/pdf/1210.5644.pdf).

<img src="https://miro.medium.com/max/2026/1*-lgwBj7KTh8His-4zMa17Q.png" alt="CRF's" width=600>